# Environment Setup

In [ ]:
import math
import os
import sys

import matplotlib.pyplot as plt
import numpy as np
import open3d as o3d

# Python version
assert sys.version_info >= (3, 8)
# Open3D version
assert o3d.__version__ >= "0.17.0"

In [ ]:
# Point cloud file I/O
filename = "data/input/iScan-Pcd-1-1.ply"
base_name, extension = os.path.splitext(os.path.basename(filename))

output_path = os.path.join("data/output", base_name)
os.makedirs(output_path, exist_ok=True)

# 1. Point Cloud Filtering

## Step 1. Down-sampling

In [ ]:
def down_sample(input_filename, output_filename, every_k_points=10):
    # Load the input point cloud file
    pcd = o3d.t.io.read_point_cloud(input_filename)

    # Function uniformly down-samples the point cloud, evenly select 1 point for every k points
    down_sampled_pcd = pcd.uniform_down_sample(every_k_points)

    # Save the down-sampled point cloud to an output file
    o3d.t.io.write_point_cloud(output_filename, down_sampled_pcd, write_ascii=False)

In [ ]:
# Perform down sampling
down_sampled_filename = os.path.join(output_path, base_name + " - downsampled" + extension)
down_sample(filename, down_sampled_filename)

## Step 2. Outlier removal

In [ ]:
# Input point cloud file
down_sampled_filename = os.path.join(output_path, base_name + " - downsampled" + extension)

In [ ]:
def outlier_removal(input_filename, output_filename, nb_neighbors=20, std_ratio=2.0):
    # Load the input point cloud file
    pcd = o3d.t.io.read_point_cloud(input_filename)

    # Statistical outlier removal
    filtered_pcd, mask = pcd.remove_statistical_outliers(nb_neighbors, std_ratio)

    # Color outliers in red
    outlier = pcd.select_by_mask(mask, invert=True)
    outlier.paint_uniform_color([1.0, 0.0, 0.0])
    print(f"Remove {outlier.point.positions.shape[0]} outliers.")

    # Save the outlier for comparison
    outlier_filename = os.path.join(output_path, base_name + " - outlier" + extension)
    o3d.t.io.write_point_cloud(outlier_filename, outlier, write_ascii=False)

    # Save the filtered point cloud to an output file
    o3d.t.io.write_point_cloud(output_filename, filtered_pcd, write_ascii=False)

In [ ]:
# Perform outlier removal
filtered_filename = os.path.join(output_path, base_name + " - filtered" + extension)
outlier_removal(down_sampled_filename, filtered_filename)

# 2. Point Cloud Segmentation

## Step 1. DBSCAN clustering

In [ ]:
# Input point cloud file
filtered_filename = os.path.join(output_path, base_name + " - filtered" + extension)

In [ ]:
from collections import Counter


def dbscan_clustering(input_filename, output_filename, eps=0.10, min_points=5):
    # Load the input point cloud file
    pcd = o3d.t.io.read_point_cloud(input_filename)
    print(pcd.point, '\n')

    # Mask by intensity to acquire low intensity points
    intensity_threshold = 50
    mask = np.where(pcd.point.intensity.numpy().flatten() <= intensity_threshold, True, False)

    # Convert mask to index
    index_of_mask = [i for i, boolean_val in enumerate(mask) if boolean_val]

    # Extract low intensity points, then perform DBSCAN on them
    low_intensity_pcd = pcd.select_by_mask(mask)
    labels = low_intensity_pcd.cluster_dbscan(eps, min_points)
    labels = labels.numpy()

    n_clusters = labels.max() + 1
    print(f"DBSCAN clustering return {n_clusters} clusters.\n")

    # Clusters will be labeled in a way that cluster with the most points is labeled 1
    if n_clusters >= 2:
        counter = Counter(labels)

        # Remove noise that be labeled -1
        del counter[-1]

        # Sort by count
        sorted_items = sorted(counter.items(), key=lambda x: x[1], reverse=True)
        rank = [item[0] for item in sorted_items]

        # Add cluster attribute to the input point cloud
        cluster = np.zeros(len(mask)).astype(np.int32)
        for index, val in enumerate(labels):
            if val != -1:
                cluster[index_of_mask[index]] = rank.index(val) + 1
        pcd.point.cluster = np.reshape(cluster, (len(cluster), 1))
        print(pcd.point)

        # Save the clustered point cloud to an output file
        o3d.t.io.write_point_cloud(output_filename, pcd, write_ascii=False)

    else:
        print("Warning: DBSCAN clustering return less than 2 clusters.")

In [ ]:
# Perform DBSCAN clustering
clustered_filename = os.path.join(output_path, base_name + " - clustered" + extension)
dbscan_clustering(filtered_filename, clustered_filename)

## Step 2. Curve fitting

In [ ]:
# Input point cloud file
clustered_filename = os.path.join(output_path, base_name + " - clustered" + extension)

In [ ]:
from scipy import interpolate
from scipy.spatial import cKDTree

# Curve information is saved for coordinate transformation
curve_point_centre = []  # B-spline representation of fitted centre curve


def curve_fitting(input_filename):
    global curve_point_centre

    # Load the input point cloud file
    pcd = o3d.t.io.read_point_cloud(input_filename)

    # Step 1. Fit two curves on both left and right rails
    # Use cluster attribute to extract rail points
    cluster = pcd.point.cluster.numpy().flatten()
    left_rail_mask = (cluster == 1)
    right_rail_mask = (cluster == 2)

    left_rail = pcd.select_by_mask(left_rail_mask)
    point_left = left_rail.point.positions.numpy()
    point_num_left = point_left.shape[0]
    print("Left rail points:", point_num_left)

    right_rail = pcd.select_by_mask(right_rail_mask)
    point_right = right_rail.point.positions.numpy()
    point_num_right = point_right.shape[0]
    print("Right rail points:", point_num_right, '\n')

    scalar_factor = point_num_left / point_num_right
    print("scalar_factor:", scalar_factor, '\n')

    point_left = np.transpose(point_left)  # (n, 3) -> (3, n)
    point_right = np.transpose(point_right)

    # Find the B-spline representation of an 3-D curve
    tck_left, u_left = interpolate.splprep(point_left)
    tck_right, u_right = interpolate.splprep(point_right)

    u_left_prime = np.linspace(u_left.min(), u_left.max(), point_num_left)
    knots_left = interpolate.splev(u_left_prime, tck_left)
    curve_point_left = np.column_stack((knots_left[0], knots_left[1], knots_left[2]))

    u_right_prime = np.linspace(u_right.min(), u_right.max(), point_num_right)
    knots_right = interpolate.splev(u_right_prime, tck_right)
    curve_point_right = np.column_stack((knots_right[0], knots_right[1], knots_right[2]))

    # Step 2. Calculate the centre line, then fit a curve on it
    # Build a kd-tree from the left curve points
    kdtree = cKDTree(curve_point_left)

    # Query the kd-tree to find the nearest neighbor and its distance
    _, nearest_index = kdtree.query(curve_point_right)

    # Get the nearest point from the left curve points
    nearest_point = curve_point_left[nearest_index]

    # Points on the centre line
    point_num_centre = point_num_right
    point_centre = (nearest_point + curve_point_right) / 2
    point_centre = np.transpose(point_centre)  # (n, 3) -> (3, n)

    # Fit a curve on centre line
    tck_centre, u_prime = interpolate.splprep(point_centre)

    u_centre_prime = np.linspace(u_prime.min(), u_prime.max(), point_num_right)
    knots_centre = interpolate.splev(u_centre_prime, tck_centre)
    curve_point_centre = np.column_stack((knots_centre[0], knots_centre[1], knots_centre[2]))

    # Step 3. Store three fitted curves to a separate file
    point = np.vstack((curve_point_left, curve_point_right, curve_point_centre))

    colors = np.vstack((np.full((curve_point_left.shape[0], 3), [0.0, 0.0, 1.0]),  # color in blue
                        np.full((curve_point_right.shape[0], 3), [0.0, 1.0, 0.0]),  # color in green
                        np.full((curve_point_centre.shape[0], 3), [1.0, 0.0, 0.0])))  # color in red

    curve = np.vstack((np.full((curve_point_left.shape[0], 1), 1),
                       np.full((curve_point_right.shape[0], 1), 2),
                       np.full((curve_point_centre.shape[0], 1), 3))).astype(np.uint8)

    pcd_curve = o3d.t.geometry.PointCloud()
    pcd_curve.point.positions = point
    pcd_curve.point.colors = colors
    pcd_curve.point.curve = curve

    # Save the fitted curves for comparison
    curve_filename = os.path.join(output_path, base_name + " - curve" + extension)
    o3d.t.io.write_point_cloud(curve_filename, pcd_curve, write_ascii=False)

    return tck_centre, u_centre_prime

In [ ]:
tck_curve, u_curve = curve_fitting(clustered_filename)  # tck, u
print("Centre curve parameters:", end=" ")
print(f"tck_curve[0] - t: {tck_curve[0]}\n")
print(f"tck_curve[1] - c: {np.array(tck_curve[1])}\n")
print(f"tck_curve[2] - k: {tck_curve[2]}\n")
print(f"u_curve: {len(u_curve)} {u_curve}")

## 3. Cross-Section Extraction

In [ ]:
def segment_ground(input_filename, distance_threshold=0.05, ransac_n=3, num_iterations=1000):
    pcd = o3d.t.io.read_point_cloud(input_filename)
    plane_model, _ = pcd.segment_plane(distance_threshold, ransac_n, num_iterations)
    return plane_model

In [ ]:
a, b, c, d = plane = segment_ground(clustered_filename).numpy()  # ax + by + cz + d = 0
print("plane parameters:", plane)

## Step 1. Coordinate Transformation

In [ ]:
# Calculate coordinates in the new coordinate system
def coordinate_transformation(input_filename, output_filename):
    global curve_point_centre

    # Load the input point cloud file
    pcd = o3d.t.io.read_point_cloud(input_filename)

    # Coordinates in original coordinate system
    coordinates = pcd.point.positions.numpy()

    # Prepare for coordinate calculation
    normal_z = np.array([a, b, c])
    if np.dot(normal_z, curve_point_centre[0]) + d < 0:  # centre curve points should have z > 0
        normal_z = -normal_z
    magnitude_z = np.linalg.norm(normal_z)

    normal_y = np.diff(curve_point_centre, axis=0)
    normal_y = np.insert(normal_y, 0, [0., 0., 0.], axis=0)
    distances = np.linalg.norm(normal_y, axis=1)
    cumulate_y = np.cumsum(distances)

    print(f"curve_point_centre: {curve_point_centre.shape}\n{curve_point_centre[:3]}\n")
    print(f"normal_y: {normal_y.shape}\n{normal_y[:3]}\n")
    print(f"distances: {distances.shape}\n{distances[:3]}\n")
    print(f"cumulate_y: {cumulate_y.shape}\n{cumulate_y[:3]}\n")

    kdtree_centre = cKDTree(curve_point_centre)
    distance, nearest_index = kdtree_centre.query(coordinates)
    nearest_point = curve_point_centre[nearest_index]

    PQ = coordinates - nearest_point
    magnitude_PQ = distance
    cross_product = np.cross(normal_z, PQ)
    sin_theta = np.linalg.norm(cross_product, axis=1) / (magnitude_z * magnitude_PQ)

    dot_product = np.sum(cross_product * normal_y[nearest_index], axis=1)
    sign_x = np.where(dot_product > 0, 1, -1)

    print(f"cross_product: {cross_product.shape}\n{cross_product[:3]}\n")
    print(f"normal_y: {normal_y.shape}\n{normal_y[:3]}\n")
    print(f"dot_product: {dot_product.shape}\n{dot_product[:3]}\n")

    # Coordinate calculation
    x = magnitude_PQ * sin_theta * sign_x
    y = cumulate_y[nearest_index]
    z = (np.dot(coordinates, normal_z) + d) / magnitude_z

    pcd.point.positions = np.column_stack((x, y, z))

    o3d.t.io.write_point_cloud(output_filename, pcd, write_ascii=False)

In [ ]:
# Perform coordinate transformation
transformed_filename = os.path.join(output_path, base_name + " - transformed" + extension)
coordinate_transformation(clustered_filename, transformed_filename)

## Step 2. Crop point cloud

In [ ]:
# Input point cloud file
transformed_filename = os.path.join(output_path, base_name + " - transformed" + extension)

In [ ]:
def crop(input_filename, output_filename):
    # Load the input point cloud file
    pcd = o3d.t.io.read_point_cloud(input_filename)

    # Geometric constraint to crop track region
    x_min, x_max = [-4.0, 8.0]
    y_min, y_max = [1e-1, math.inf]
    z_min, z_max = [-math.inf, 1.0]

    # Create bounding box
    min_bound = np.array([x_min, y_min, z_min])
    max_bound = np.array([x_max, y_max, z_max])
    bounding_box = o3d.t.geometry.AxisAlignedBoundingBox(min_bound, max_bound)

    # Crop the point cloud
    pcd = pcd.crop(bounding_box)

    o3d.t.io.write_point_cloud(output_filename, pcd, write_ascii=False)

In [ ]:
# Perform crop point cloud
cropped_filename = os.path.join(output_path, base_name + " - cropped" + extension)
crop(transformed_filename, cropped_filename)

## Step 3. Split point cloud

In [ ]:
# Input point cloud file
cropped_filename = os.path.join(output_path, base_name + " - cropped" + extension)

In [ ]:
def axis_downsampling(data, delta_x=0.001):
    # Sort the data points based on x-values
    sorted_indices = np.argsort(data[:, 0])
    sorted_data = data[sorted_indices]

    # Calculate the number of intervals
    num_intervals = int(np.ceil((sorted_data[-1, 0] - sorted_data[0, 0]) / delta_x))

    downsampled_data = []

    interval_start = sorted_data[0, 0]
    interval_end = interval_start + delta_x

    current_sum = 0
    num_points = 0

    # Iterate over sorted data points and perform downsampling
    for i in range(len(sorted_data)):
        if sorted_data[i, 0] < interval_end:
            current_sum += sorted_data[i, 1]
            num_points += 1
        else:
            if num_points != 0:
                downsampled_data.append([interval_start, current_sum / num_points])

            # move to next interval
            while not (sorted_data[i, 0] < interval_end):
                interval_end += delta_x
            interval_start = interval_end - delta_x

            current_sum = sorted_data[i, 1]
            num_points = 1

    # Sort the data points based on x-values
    downsampled_data = np.array(downsampled_data)
    sorted_indices = np.argsort(downsampled_data[:, 0])
    downsampled_data = downsampled_data[sorted_indices]

    return downsampled_data

In [ ]:
import bisect

from matplotlib import ticker
from scipy.interpolate import griddata
from scipy.interpolate import splprep, splev


def format_axes(ax):
    ax.xaxis.set_minor_locator(ticker.AutoMinorLocator())
    ax.xaxis.set_major_formatter('{:.1f}'.format)
    ax.yaxis.set_minor_locator(ticker.AutoMinorLocator())
    ax.yaxis.set_major_formatter('{:.1f}'.format)


# Split the point cloud, then draw a depth image and a cross-section image on each slice
def split(input_filename, every_k_meter=10):
    # Load the input point cloud file
    pcd = o3d.t.io.read_point_cloud(input_filename)

    x_max_global, y_max_global, z_max_global = pcd.get_max_bound().numpy()
    x_min_global, y_min_global, z_min_global = pcd.get_min_bound().numpy()
    print(f"Total length: {y_max_global:.2f} meters\n")

    y_split_pos = [0]
    while y_split_pos[-1] < y_max_global:
        y_split_pos.append(y_split_pos[-1] + every_k_meter)

    y_boundary = []
    for i in range(len(y_split_pos) - 1):
        boundary = [y_split_pos[i], y_split_pos[i + 1]]
        y_boundary.append(boundary)
    print(f"Split the point cloud into {len(y_boundary)} slices: {y_boundary}\n")

    slices_path = os.path.join(output_path, base_name + " - slices")
    os.makedirs(slices_path, exist_ok=True)

    pcd_path = os.path.join(slices_path, "point cloud")
    os.makedirs(pcd_path, exist_ok=True)

    image_path = os.path.join(slices_path, "images")
    os.makedirs(image_path, exist_ok=True)

    for y_min_slice, y_max_slice in y_boundary:
        # Crop point cloud
        min_bound = np.array([x_min_global, y_min_slice, z_min_global])
        max_bound = np.array([x_max_global, y_max_slice, z_max_global])
        bounding_box = o3d.t.geometry.AxisAlignedBoundingBox(min_bound, max_bound)
        pcd_slice = pcd.crop(bounding_box)

        # Slice number
        slice_number = y_max_slice // 10
        slice_base_name = base_name + " - slice {}".format(slice_number)

        # Extract attributes
        intensity = pcd_slice.point.intensity.numpy().flatten()
        cluster = pcd_slice.point.cluster.numpy().flatten()
        left_rail_mask = (cluster == 1)
        right_rail_mask = (cluster == 2)

        # Extract points
        coordinates = pcd_slice.point.positions.numpy()
        x, y, z = coordinates[:, 0], coordinates[:, 1], coordinates[:, 2]
        x_min, y_min, z_min = pcd_slice.get_min_bound().numpy()
        x_max, y_max, z_max = pcd_slice.get_max_bound().numpy()
        point_left = coordinates[left_rail_mask]
        point_right = coordinates[right_rail_mask]

        # Create a figure and define the grid layout
        fig = plt.figure(figsize=(16, 9))  # Set the overall figure size
        fig.suptitle("Slice Profile — " + base_name, fontsize=14)
        gs = fig.add_gridspec(8, 2)  # Define a grid for subplots

        # Create subplots
        ax_point = fig.add_subplot(gs[:4, 0], projection="3d")
        ax_cross = fig.add_subplot(gs[4:6, 0])
        ax_cross_prime = fig.add_subplot(gs[6:, 0])
        ax_depth = fig.add_subplot(gs[:7, 1])
        ax_text = fig.add_subplot(gs[7, 1])

        # 1. Point cloud (left-top, 3D subplot)
        # Axes setting
        ax_point.set(xlabel="x axis", ylabel="y axis", zlabel="z axis", zticks=[])
        ax_point.set_title(f"Transformed Point Cloud — Slice {slice_number}", fontsize=12)
        ax_point.set_box_aspect([np.ptp(x), np.ptp(y), np.ptp(z)])
        ax_point.view_init(elev=45, azim=-115)

        # Plotting data
        ax_point.scatter3D(x, y, z, s=0.01, marker=',', c=intensity, cmap="gray")  # points
        ax_point.plot([0.0, 0.0], [y_min, y_max], [1.0, 1.0], label="Centre line",
                      color='r', zorder=10)
        ax_point.plot(point_left[:, [0]], point_left[:, [1]], point_left[:, [2]], label="Left rail",
                      marker=',', markersize=10, color="cornflowerblue", zorder=10)
        ax_point.plot(point_right[:, [0]], point_right[:, [1]], point_right[:, [2]], label="Right rail",
                      marker=',', markersize=10, color="limegreen", zorder=10)
        ax_point.legend()

        # 2. Depth images (right, 2D subplot)
        # Axes setting
        ax_depth.set(xlabel="width (m)", ylabel="mileage (m)")
        ax_depth.set_title("Depth Image", fontsize=12)
        format_axes(ax_depth)

        # Define the grid on which to interpolate the points
        x_threshold = [x_min_global, x_max_global]
        if slice_number != len(y_boundary):  # not the last slice
            y_threshold = [y_min_slice, y_max_slice]
        else:
            y_threshold = [y_min, y_max]
        ax_depth.set_box_aspect(np.ptp(y_threshold) / np.ptp(x_threshold))  # Axis ratio is fixed

        # Interpolate the points onto the grid
        grid_x, grid_y = np.mgrid[x_threshold[0]:x_threshold[1]:1200j, y_threshold[0]:y_threshold[1]:1000j]
        grid_z = griddata((x, y), z, (grid_x, grid_y), method="nearest")

        # Plot the interpolated grid using pcolormesh
        z_threshold = [-1.0, 0.3]
        pc = ax_depth.pcolormesh(grid_x, grid_y, grid_z, vmin=z_threshold[0], vmax=z_threshold[1], cmap="RdBu_r")
        fig.colorbar(pc, ax=ax_depth, extend="max")

        # 3. Cross-section image (left-bottom, 2D subplot)
        rail_height = 0.176  # 176 mm
        rail_head_width = 0.073  # 73 mm
        sleeper_length = 2.60  # 2600 mm
        sleeper_height = 0.230  # 230 mm
        half_sleeper_length = sleeper_length / 2

        ideal_top_width = 3.1  # 3.1 m
        ideal_slope = 1 / 1.75  # 1:1.75
        half_ideal_top_width = ideal_top_width / 2

        # Color points
        points = np.column_stack((x, z))
        interval_endpoint = [-3.0, -1.8, -half_sleeper_length, half_sleeper_length, 1.8, 3.0]

        left_remainder = points[points[:, 0] <= interval_endpoint[0]]
        left_area = points[(points[:, 0] > interval_endpoint[0]) & (points[:, 0] < interval_endpoint[2])]
        sleeper_area = points[(points[:, 0] >= interval_endpoint[2]) & (points[:, 0] <= interval_endpoint[3])]
        right_area = points[(points[:, 0] > interval_endpoint[3]) & (points[:, 0] < interval_endpoint[5])]
        right_remainder = points[points[:, 0] >= interval_endpoint[5]]

        # Compute the left shoulder point first
        max_z_index = np.argmax(left_area[:, 1])
        max_point = left_area[max_z_index]
        interval_endpoint[1] = max_point[0]

        # Then use interval endpoints to separate regions
        left_slope = left_area[left_area[:, 0] <= interval_endpoint[1]]
        left_shoulder = left_area[left_area[:, 0] > interval_endpoint[1]]

        right_shoulder = right_area[right_area[:, 0] < interval_endpoint[4]]
        right_slope = right_area[right_area[:, 0] >= interval_endpoint[4]]

        # 3.1 Cross-section image (left-bottom 1)
        # Axes setting
        ax_cross.set(ylabel="Z")
        ax_cross.set_title("Cross-section Image", loc="left", fontsize=12)
        ax_cross.set_title("Inspection Profile")
        format_axes(ax_cross)

        # Plotting data
        regions = [sleeper_area, left_slope, left_shoulder, right_shoulder]
        labels = ["Sleeper region", "Left slope", "Left shoulder", "Right shoulder"]
        colors = ["red", "cornflowerblue", "orange", "limegreen"]
        for i in range(len(regions)):
            ax_cross.scatter(regions[i][:, 0], regions[i][:, 1], marker='^', s=0.01, c=colors[i], label=labels[i])

        point_on_top = [max_point[0], -half_sleeper_length, half_sleeper_length, interval_endpoint[4]]
        colors = ["cornflowerblue", "red", "red", "limegreen"]
        for i in range(len(point_on_top)):
            ax_cross.plot([point_on_top[i], point_on_top[i]], [0, -1.0], linestyle='--', c=colors[i])

        remainder = np.vstack((left_remainder, right_remainder, right_slope))
        ax_cross.scatter(remainder[:, 0], remainder[:, 1], s=0.01, marker='^', c="gray")

        z_threshold = [-1.0, 1.0]  # different from depth image
        ax_cross.set_xlim(x_threshold)
        ax_cross.set_ylim(z_threshold)
        ax_cross.set_box_aspect(np.ptp(z_threshold) / np.ptp(x_threshold))
        ax_cross.set_yticks([-1.0, 0.0, 1.0])

        ax_cross.legend(markerscale=50)

        # 3.2 Cross-section image (left-bottom 2)
        # Axes setting
        ax_cross_prime.set(xlabel="X", ylabel="Z")
        ax_cross_prime.set_title("Comparison Diagram")
        format_axes(ax_cross_prime)

        # Plotting data
        delta_x = 0.001
        ballast_bed_points = np.vstack((left_slope, left_shoulder, sleeper_area, right_shoulder, right_slope))
        ballast_bed_points = axis_downsampling(ballast_bed_points, delta_x)

        # Fit a B-spline curve to the points
        tck, u = splprep(ballast_bed_points.T, s=0.4)
        u_prime = np.linspace(u.min(), u.max(), 1000)
        x_prime, y_prime = splev(u_prime, tck)

        # Plot ideal section and actual section
        ax_cross_prime.plot([-half_ideal_top_width, half_ideal_top_width], [0, 0], c="red", label="Ideal section")
        ax_cross_prime.plot(x_prime, y_prime, c="limegreen", label="Actual section", zorder=10)

        width_offset = 2
        height_offset = ideal_slope * width_offset
        ax_cross_prime.plot([-half_ideal_top_width, -(half_ideal_top_width + width_offset)],  # ideal left slope
                            [0, -height_offset], c="red")
        ax_cross_prime.plot([half_ideal_top_width, half_ideal_top_width + width_offset],  # ideal right slope
                            [0, -height_offset], c="red")

        # Plot vertical lines
        point_on_top = [-half_sleeper_length, half_sleeper_length]
        for point in point_on_top:
            ax_cross_prime.plot([point, point], [0, -1.0], linestyle='--', c='r')
        ax_cross_prime.plot([0, 0], [0, -1.0], linestyle='--', c='k')

        # Left shoulder point
        index = bisect.bisect_left(ballast_bed_points[:, 0], max_point[0])
        max_point_prime = ballast_bed_points[index]

        ax_cross_prime.plot([max_point_prime[0]], [max_point_prime[1]], marker='o', c="cornflowerblue", zorder=20)
        ax_cross_prime.plot([max_point_prime[0], max_point_prime[0]], [max_point_prime[1], -1.0], linestyle='--',
                            c="cornflowerblue", zorder=20)
        ax_cross_prime.annotate("Left shoulder",
                                xy=(max_point_prime[0] - 0.05, max_point_prime[1] + 0.05),
                                xytext=(-2.8, 0.25),
                                arrowprops=dict(facecolor="cornflowerblue", headwidth=10, headlength=10))

        ax_cross_prime.legend(loc="lower right")

        x_threshold_prime = [-3.0, 3.0]
        z_threshold_prime = [-1.0, 0.5]
        ax_cross_prime.set_xlim(x_threshold_prime)
        ax_cross_prime.set_ylim(z_threshold_prime)
        ax_cross_prime.set_box_aspect(np.ptp(z_threshold_prime) / np.ptp(x_threshold_prime))
        ax_cross_prime.set_yticks([-1.0, -0.5, 0.0, 0.5])

        # 4. Statistic data
        # Linear fitting
        coefficients = np.polyfit(left_slope[:, 0], left_slope[:, 1], 1)
        slope, y_intercept = coefficients[0], coefficients[1]

        left_slope_value = 1 / slope
        left_shoulder_width = (-max_point[0] - half_sleeper_length) * 1000  # mm
        # ideal_shoulder_width = (half_ideal_top_width - half_sleeper_length) * 1000

        text_left = f"""
        Left  slope — 1 : {left_slope_value:.2f}\n
        Ideal slope — 1 : 1.75\n
        """

        text_right = f"""
        Left  shoulder width — {left_shoulder_width:.0f} mm\n
        Ideal shoulder width — 450 mm\n
        """

        ax_text.text(0, 0.5, text_left, ha="left", va="center", fontsize=12)
        ax_text.text(0.4, 0.5, text_right, ha="left", va="center", fontsize=12)

        ax_text.set_xlim(0, 1)
        ax_text.set_ylim(0, 1)

        ax_text.axis("off")

        # Plotting complete, save the image
        plt.tight_layout()

        image_filename = os.path.join(image_path, slice_base_name + ".png")
        fig.savefig(image_filename, format="png", dpi=300)

        # Save the split point cloud
        pcd_filename = os.path.join(pcd_path, slice_base_name + extension)
        o3d.t.io.write_point_cloud(pcd_filename, pcd_slice, write_ascii=False)

In [ ]:
# Perform split point cloud
split(cropped_filename)